## Домашнее задание к лекции "Основы веб-скрапинга и работы с API"

### Задание 1.

#### Обязательная часть
Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

In [9]:
import requests

import time

import pandas as pd

import datetime

In [74]:
KEYWORDS = ['python', 'парсинг', 'Athelas', 'LinkedIn']

In [4]:
from bs4 import BeautifulSoup

In [17]:
def change_str_habr_date(str_date):
    import datetime
    
    months = {'1':'января', '2':'февраля','3':'марта','4':'апреля','5':'мая','6':'июня','7':'июля','8':'августа','9':'сентября','10':'октября','11':'ноября','12':'декабря'}
    date_1 = 'сегодня'
    date_2 = 'вчера'
    
    if date_1 in str_date:
        time = datetime.datetime.strptime(str_date, 'сегодня в %H:%M').time()
        date = datetime.datetime.today().date()
        date_post = datetime.datetime.combine(date, time)
    elif date_2 in str_date:
        time = datetime.datetime.strptime(str_date, 'вчера в %H:%M').time()
        date = datetime.datetime.now().date() - datetime.timedelta(days=1) 
        date_post = datetime.datetime.combine(date, time)
    else:
        for key, values in months.items():
            if values in str_date:
                str_date = str_date.replace(values, key)
                date_post = datetime.datetime.strptime(str_date, '%d %m %Y в %H:%M')
    
    return date_post
                

In [78]:
URL = 'https://habr.com/ru/all/'
res = requests.get(URL)
soup = BeautifulSoup(res.text, 'html.parser')
posts = soup.find_all('article', class_='post')

row = dict()
habr_news = pd.DataFrame()

for post in posts:
    time.sleep(0.3)
    post_class_link = post.find('h2', class_='post__title')
    post_link =  post_class_link.find('a').get('href')
    title_post = post_class_link.text
    text_preview = post.find('div', class_='post__text').text
    text_date = post.find('span', class_='post__time').text
    date = change_str_habr_date(text_date)
    for word in KEYWORDS:
        if word in title_post or word in text_preview:
            row = {'date': date, 'link':post_link, 'title':title_post, 'preview':text_preview}
            habr_news = pd.concat([habr_news, pd.DataFrame([row])])

habr_news

,date,link,title,preview
0,2020-12-04 13:54:00,https://habr.com/ru/post/531322/,\nДиагностика крови с помощью Deep Learning: К...,"\n\n\nСлева — Танай Тандон, в 17 лет в 2014 го..."
0,2020-12-04 12:43:00,https://habr.com/ru/post/531304/,\nКак в LinkedIn настроили программу удаленног...,"\nНашел любопытный кейс, как в LinkedIn перест..."


#### Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

In [ ]:
# получаем часть HTML кода по ссылке (переменная post_link)

# soup_for_full_text = BeautifulSoup(requests.get(post_link).text, 'html.parser')
# soup_for_full

In [89]:
row_full = dict()
habr_news_full = pd.DataFrame()

for post in posts:
    time.sleep(0.3)
    post_class_link = post.find('h2', class_='post__title')
    post_link =  post_class_link.find('a').get('href')
    title_post = post_class_link.text
    soup_for_full_text = BeautifulSoup(requests.get(post_link).text, 'html.parser')
    text = soup_for_full_text.find('div', class_='post__text').text
    text_date = post.find('span', class_='post__time').text
    date = change_str_habr_date(text_date)
    for word in KEYWORDS:
        if word in title_post or word in text:
            row_full = {'date': date, 'link':post_link, 'title':title_post, 'text':text}
            habr_news_full = pd.concat([habr_news_full, pd.DataFrame([row_full])])

            
habr_news_full

,date,link,title,text
0,2020-12-04 13:54:00,https://habr.com/ru/post/531322/,\nДиагностика крови с помощью Deep Learning: К...,"\n\nСлева — Танай Тандон, в 17 лет в 2014 году..."
0,2020-12-04 12:43:00,https://habr.com/ru/post/531304/,\nКак в LinkedIn настроили программу удаленног...,"Нашел любопытный кейс, как в LinkedIn перестро..."


### Задание 2.

#### Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [6]:
import json

In [3]:
URL_1 = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
headers = {'Vaar-Version': '0', 'Vaar-Header-App-Product':'hackcheck-web-avast'}

In [4]:
emailAddresses = ['amoshkov@mail.ru', 'aamoshkov@gmail.com']
dict_data = dict()
dict_data['emailAddresses'] = emailAddresses

In [7]:
data = json.dumps(dict_data)

In [10]:
res = requests.post(URL_1, json=dict_data, headers=headers)
dict_res = res.json()

In [11]:
leak_email = pd.DataFrame()

for email in emailAddresses:
    k = len(dict_res['summary'][f'{email}']['breaches'])
    for i in range(k):
        a = dict_res['summary'][f'{email}']['breaches'][i]
        site = dict_res['breaches'][f'{a}']['site']
        text = dict_res['breaches'][f'{a}']['description']
        date = dict_res['breaches'][f'{a}']['publishDate']
        row = {'email': email, 'date': date, 'source':site, 'description': text}
        leak_email = pd.concat([leak_email, pd.DataFrame([row])])
        
        
leak_email

,email,date,source,description
0,amoshkov@mail.ru,2020-05-21T00:00:00Z,vk.com,"At some time in 2020, the Russian social netwo..."
0,amoshkov@mail.ru,2016-12-15T00:00:00Z,vedomosti.ru,"In May 2012, Russian business newspaper Vedomo..."
0,amoshkov@mail.ru,2018-06-19T00:00:00Z,storelp.ru,"At an unconfirmed date, StoreLP.ru's database ..."
0,amoshkov@mail.ru,2020-07-16T00:00:00Z,unknown,"At an unconfirmed date, data purportedly assoc..."
0,amoshkov@mail.ru,2017-03-31T00:00:00Z,fc-zenit.ru,"In July 2010, FC Zenit's user database was all..."


#### Дополнительная часть (необязательная)

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.
Документация к API VK: https://vk.com/dev/methods , вам поможет метод wall.get

GROUP = 'netology'  
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ
В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>

#### ПРИМЕЧАНИЕ
Домашнее задание сдается ссылкой на репозиторий GitHub. Не сможем проверить или помочь, если вы пришлете:

файлы;
архивы;
скриншоты кода.
Все обсуждения и консультации по выполнению домашнего задания ведутся только на соответствующем канале в slack.

##### Как правильно задавать вопросы аспирантам, преподавателям и коллегам?

Прежде чем задать вопрос необходимо попробовать найти ответ самому в интернете. Навык самостоятельного поиска информации – один из важнейших, и каждый практикующий специалист любого уровня это делает каждый день.

Любой вопрос должен быть сформулирован по алгоритму:
1) Что я делаю?
2) Какого результата я ожидаю?
3) Как фактический результат отличается от ожидаемого?
4) Что я уже попробовал сделать, чтобы исправить проблему?

По возможности, прикрепляйте к вопросу скриншоты, либо ссылки на код. Оставляйте только проблемный и воспроизводимый участок кода, все решение выкладывать не допускается.